In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from networkx.utils.random_sequence import (powerlaw_sequence)

# import networkx.utils.create_degree_sequence as create_degree_sequence
%matplotlib notebook

In [ ]:
def create_degree_sequence(n, sfunction=None, max_tries=50, **kwds):
    """ Attempt to create a valid degree sequence of length n using
    specified function sfunction(n,**kwds).

    Parameters
    ----------
    n : int
        Length of degree sequence = number of nodes
    sfunction: function
        Function which returns a list of n real or integer values.
        Called as "sfunction(n,**kwds)".
    max_tries: int
        Max number of attempts at creating valid degree sequence.

    Notes
    -----
    Repeatedly create a degree sequence by calling sfunction(n,**kwds)
    until achieving a valid degree sequence. If unsuccessful after
    max_tries attempts, raise an exception.
    
    For examples of sfunctions that return sequences of random numbers,
    see networkx.Utils.

    Examples
    --------
    >>> from networkx.utils import uniform_sequence, create_degree_sequence
    >>> seq=create_degree_sequence(10,uniform_sequence)
    """
    tries=0
    max_deg=n
    while tries < max_tries:
        trialseq=sfunction(n,**kwds)
        # round to integer values in the range [0,max_deg]
        seq=[min(max_deg, max( int(round(s)),0 )) for s in trialseq]
        # if graphical return, else throw away and try again
        if nx.is_graphical(seq):
            return seq
        tries+=1
    raise nx.NetworkXError(\
          "Exceeded max (%d) attempts at a valid sequence."%max_tries)

In [ ]:
#### code from http://nbviewer.jupyter.org/gist/Midnighter/248f1a5d8c21b39525ae

exp = 2.9
seed = 1223445
num = 1000
print("exponent = -{0:.2f}".format(exp))
sequence = create_degree_sequence(num, powerlaw_sequence, exponent=exp)
graph = nx.configuration_model(sequence, seed=seed)
# count parallel edges and avoid counting A-B as well as B-A
num_par = sum(len(graph[node][neigh]) for node in graph for neigh in graph.neighbors(node)) // 2
print("graph has {0:d} parallel edges".format(num_par))
loops = graph.selfloop_edges()
# remove them
graph = nx.Graph(graph)
graph.remove_edges_from(loops)
# get largest connected component
# unfortunately, the iterator over the components is not guaranteed to be sorted by size
components = sorted(nx.connected_components(graph), key=len, reverse=True)
lcc = graph.subgraph(components[0])
print("size of largest connected component = {0:d}".format(len(lcc)))
# new degree sequence
simple_seq = [deg for (node, deg) in lcc.degree()]
# create histograms
counts = np.bincount(sequence)
mask = (counts > 0)
plt.figure()
plt.plot(np.arange(len(counts))[mask], counts[mask] / counts.sum(), "o", label="MultiGraph")
simple_counts = np.bincount(simple_seq)
mask = (simple_counts > 0)
# distribution is shifted for visibility
plt.plot(np.arange(len(simple_counts))[mask], simple_counts[mask] / simple_counts.sum() / 10.0,
         "o", label="Simple LCC")
x = np.arange(1, len(counts))
plt.plot(x, np.power(x, -exp))
plt.xlabel(r"Degree $k$")
plt.xscale("log")
plt.ylabel(r"Probability $P(k)$")
plt.yscale("log")
plt.title(r"$N = {0:d}, \quad \lambda = {1:.2f}$".format(num, exp))
plt.legend(loc="best")
plt.show()

In [ ]:
plt.figure()
pos = nx.spring_layout(graph)
nx.draw(G, pos, node_color='b', node_size=50, with_labels=False)
plt.show()

In [ ]:
class Person():
    def __init__(self,i):
        # time_since_infection equals -1 if not infected
        self.identifier = i
        self.disease_status = 0
        self.time_since_infection = -1
    
    def __eq__(self, other):
        if isinstance(other, int):
            return self.disease_status == other

In [ ]:
print(Person(i) == 0)

for i in range(len(graph)):
    graph.node[i]["Data"] = Person(i)

In [ ]:
list(graph.nodes(data="data"))